In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [18]:
import tensorflow as tf

# Load the InceptionV3 model
inception = tf.keras.applications.InceptionV3(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

# Freeze the pre-trained layers
for layer in inception.layers:
    layer.trainable = False

# Add a new fully connected layer
x = inception.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

# Create a new model
model = tf.keras.models.Model(inputs=inception.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_282 (Conv2D)            (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_282 (Batch  (None, 111, 111, 32  96         ['conv2d_282[0][0]']             
 Normalization)                 )                                                           

In [7]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True)

In [19]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)


train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Final Year Project /forest_fire/Training and Validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Final Year Project /forest_fire/Testing',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 4453 images belonging to 2 classes.
Found 57 images belonging to 2 classes.


In [20]:

model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10
140/140 [==============================] - 678s 5s/step - loss: 0.2921 - accuracy: 0.8976 - val_loss: 0.0284 - val_accuracy: 1.0000
Epoch 2/10
140/140 [==============================] - 660s 5s/step - loss: 0.1795 - accuracy: 0.9302 - val_loss: 0.0235 - val_accuracy: 0.9825
Epoch 3/10
140/140 [==============================] - 664s 5s/step - loss: 0.1719 - accuracy: 0.9295 - val_loss: 0.0413 - val_accuracy: 0.9825
Epoch 4/10
140/140 [==============================] - 661s 5s/step - loss: 0.1372 - accuracy: 0.9443 - val_loss: 0.0174 - val_accuracy: 1.0000
Epoch 5/10
140/140 [==============================] - 666s 5s/step - loss: 0.1324 - accuracy: 0.9486 - val_loss: 0.0228 - val_accuracy: 0.9825
Epoch 6/10
140/140 [==============================] - 664s 5s/step - loss: 0.1399 - accuracy: 0.9454 - val_loss: 0.0400 - val_accuracy: 0.9825
Epoch 7/10
140/140 [==============================] - 662s 5s/step - loss: 0.1060 - accuracy: 0.9567 - val_loss: 0.0167 - val_accuracy: 1.0000

In [21]:
model.save("Fire_prediction_model_inceptionv3.h5")